In [1]:
## import time
import pymongo
import json
import pprint
from pymongo.errors import WriteError, DuplicateKeyError
from pymongo import MongoClient
from datetime import datetime
import os
client = MongoClient('mongodb://localhost:27017')
testDB = client.abaco

## Search

In [11]:
# Example request
#'?search=car&exactsearch=DEV-DEVELOP&search=dog&&skip=0&limit=10'

# Mongo aggregation comparison expression operators to implement:
# $eq, $gt, %gte, $lt, $lte, $ne, $in, $nin

def get(search_type, args):
    """
    Placeholder
    """
    # Given 
    tenant = 'DEV-DEVELOP'
    curr_user = 'testuser'
    
    search_type = search_type.lower()
    search, query, paging = arg_parser(args, tenant)
    
    # Executions and Workers store
    if search_type in ['executions', 'workers']:
        nested_search_pipeline(search_type, search, query, paging)
    elif search_type in ['actors', 'logs']:
        flat_search_pipeline(search_type, search, query, paging)
    else:
        raise KeyError(f'Inputted search_type is invalid, must be one of {list(store_dict.keys())}.')
    

def nested_search_pipeline(search_type, search, query, paging):
    store_dict = {'executions': testDB['3'],
                  'workers': testDB['6']}
    queried_store = store_dict[search_type]
        
    # First pipeline setup and work
    security = [{'$lookup':
                    {'from' : '2',
                     'localField' : '_id',
                     'foreignField' : '_id',
                     'as' : 'permissions'}},
                {'$unwind': '$permissions'},
                {'$match': {'permissions.' + curr_user: {'$exists': True}}}]
    first_pipeline = search + security
    first_res = list(queried_store.aggregate(pipeline))
    print('-----PIPELINE-----\n')
    pprint.pprint(first_pipeline)
    print('\n\n\n-----RESULTANT-----')
    pprint.pprint(first_res)

    # Second pipeline setup and work
    flat_first_res = mongo_flattener(first_res)
    ## Should implement a call to use a random/unused database.
    testDB['20'].drop()
    testDB['20'].insert_many(flat_first_res)
    testDB['20'].drop()
    
    # This should be implemented in arg_parser later
    search = [{'$match': {'executions.id': '8864362'}}]
    
    second_pipeline = search + query + security + project + paging
    second_res = list(queried_store.aggregate(pipeline))
    print('-----PIPELINE-----\n')
    pprint.pprint(second_pipeline)
    print('\n\n\n-----RESULTANT-----')
    pprint.pprint(second_res)

    
def arg_parser(args, tenant):
    """
    Placeholder
    """
    query = []
    search_terms = f"{tenant}"
    skip_amo = None
    limit_amo = None
    
    # Parsing args given by user, if multiple given, the arg comes as a list
    for key, val in args.items():
        # Adding search terms to 'search_terms'
        if key == "search":
            if isinstance(val, list):
                joined_val = ' '.join(val)
                search_terms = search_terms + f' {joined_val}'
            else:
                search_terms = search_terms + f' {val}'
        # Adding exact search terms to 'search_terms', but now with double quotes
        elif key == "exactsearch":
            if isinstance(val, list):
                joined_val = '" "'.join(val)
                search_terms = search_terms + f' "{joined_val}"'
            else:
                search_terms = search_terms + f' "{val}"'
        # Gets the number of results to skip
        elif key == "skip":
            if isinstance(val, list):
                raise KeyError('Inputted "skip" paramater is invalid, parameter can only be specified once')
            else:
                skip_amo = int(val)
        # Gets the number of results to show
        elif key == "limit":
            if isinstance(val, list):
                raise KeyError('Inputted "limit" paramater is invalid, parameter can only be specified once')
            else:
                limit_amo = int(val)
        else:
            # Checking for logical operators and adds a pipeline match if found
            used_oper = False
            for oper in ['.$eq', '.$gt', '.$gte', '.$lt', '.$lte', '.$ne']:
                if oper in key:
                    key = key.split(oper)[0]
                    oper = oper[1:]
                    query += [{'$match': {key: {oper: val}}}]
                    used_oper = True
                    break
            # If no local operators found then a regular pipeline match is used
            if not used_oper:
                query += [{'$match': {key: val}}]

    ####### Check if search should start with tenant or not for speed. If so, delete this.
    if search_terms:
        search = [{'$match': {'$text': {'$search': search_terms}}},
                  {'$sort': {'score': {'$meta': 'textScore'}}}]
    #pprint.pprint(search)
    
    paging = []
    if skip_amo:
        paging = paging + [{'$skip': skip_amo}]
    if limit_amo:
        paging = paging + [{'$limit': limit_amo}]
    #pprint.pprint(paging)

    return search, query, paging

def mongo_flattener(mongo_input):
    flatList = []
    for actor in mongo_input:
        list_of_execution_info = []
        aid = actor['_id']
        newDict = {'_id': aid}
        del actor['_id']
        
        ################################################################DELETE THIS LATER
        try:
            del actor['permissions']
        except:
            pass
        for _, value in actor.items():
            list_of_execution_info.append(value)
        newDict.update({'executions': list_of_execution_info})
        flatList.append(newDict)
    return flatList

In [12]:
# '?search=7KEaNE8lrRw5k&search=KUBS&exactsearch=car&exactsearch=POPS&aid=FORDHmongENRY&aid.$gt=100010010'
# args = {'search': ['7KEaNE8lrRw5k', 'KUBS'], 'exactsearch': ['car', 'POPS'], 'aid':'FORDHENRY', 'aid.$gt': '100010010'}

# '?search=DEV-DEVELOP&aid=FORDHENRY&aid.$gt=100010010&skip=0&limit=10'
# args = {'search': 'DEV-DEVELOP', 'aid':'FORDHENRY', 'aid.$gt': '100010010', 'skip':'0', 'limit':'10'}

args = {'search': 'DEV-DEVELOP'}

#args = {'search': 'DEV-DEVELOP_jJqjmq57Vjv55'}

#args = {'exactsearch': ''}

#args = {'search': 'DEV-DEVELOP', 'actor_id': 'DEV-DEVELOP_jJqjmq57Vjv55'}


get('executions', args)

-----PIPELINE-----

[{'$match': {'$text': {'$search': [{'$match': {'$text': {'$search': 'DEV-DEVELOP '
                                                                    'DEV-DEVELOP'}}},
                                   {'$sort': {'score': {'$meta': 'textScore'}}}]}}},
 {'$sort': {'score': {'$meta': 'textScore'}}},
 {'$lookup': {'as': 'permissions',
              'foreignField': '_id',
              'from': '2',
              'localField': '_id'}},
 {'$unwind': '$permissions'},
 {'$match': {'permissions.testuser': {'$exists': True}}}]



-----RESULTANT-----


KeyboardInterrupt: 

## Indexing stuff

In [36]:
# Do this to create a wildcard index on littttterally everything.
testDB['3'].create_index([('$**', pymongo.TEXT)])

'$**_text'

In [ ]:
list(testDB[''].list_indexes())

## Basic pipeline

In [ ]:
search = "{tenant: DEV-DEVELOP}"
pipeline = [{'$match': {'tenant': tenant}},
            {'$lookup':
                {'from' : '2',
                 'localField' : '_id',
                 'foreignField' : '_id',
                 'as' : 'permissions'}},
            {'$unwind': '$permissions'},
            {'$match': {'permissions.' + curr_user: {'$exists': True}}}]

list(testDB['3'].aggregate(pipeline))

## Complexer pipeline

In [ ]:
#ex. Update logs fields if actors exists
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}},
            {'$match': {'tenant': tenant}},
            {'$lookup':
                {'from' : '2',
                 'localField' : 'actor_id',
                 'foreignField' : '_ifd',
                 'as' : 'permissions'}},
            {'$unwind': '$permissions'},
            {'$match': {'permissions.' + curr_user: {'$exists': True}}},
            {'$skip': skip_amo},
            {'$limit': limit_amo}]
            #{'project': {ijaiosjdioasjd}}

db = '2' 
    
pipeline = [{'$lookup':
                {'from' : '5', #Actors_db
                 'localField' : '_id',
                 'foreignField' : '_id',
                 'as' : 'actor_dict'}},
            {'$unwind': '$actor_dict'},
           ]

list(testDB['3'].aggregate(pipeline))

## Complex executions pipeline

In [3]:
def converting_first_result(first_stage_json):
    newList = []
    for actor in first_stage_json:
        list_of_execution_info = []
        aid = actor['_id']
        newDict = {'_id': aid}
        del actor['_id']
        try:
            del actor['permissions']
        except:
            pass
        for key, value in actor.items():
            list_of_execution_info.append(value)
        newDict.update({'executions': list_of_execution_info})
        newList.append(newDict)
    return newList

In [4]:
tenant = 'DEV-DEVELOP'
curr_user = 'testuser'

In [20]:
search = f'DEV-DEVELOP'
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}}]
old_json = list(testDB['5'].aggregate(pipeline))

In [18]:
old_json

[]

In [5]:
# Executions
# First Pipeline
search = '"DEV-DEVELOP_1952559"'
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}}]
old_json = list(testDB['3'].aggregate(pipeline))

In [6]:
second_stage_input = converting_first_result(old_json)

In [ ]:
testDB['20'].drop()
start = time.time()

testDB['20'].insert_many(second_stage_input)
print(time.time() - start)

In [ ]:
import time
start = time.time()
pipeline = [{'$match': {'executions.id': '8864362'}}]

old_json = list(testDB['3'].aggregate(pipeline))
print(time.time() - start)


In [ ]:
# Executions
# First Pipeline
search = '"DEV-DEVELOP_1952559"'
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}}]
old_json = list(testDB['3'].aggregate(pipeline))

second_stage_input = converting_first_result(old_json)

testDB['20'].drop()
start = time.time()

testDB['20'].insert_many(second_stage_input)
print(time.time() - start)

import time
start = time.time()
pipeline = [{'$match': {'executions.id': '8864362'}}]

old_json = list(testDB['3'].aggregate(pipeline))
print(time.time() - start)


## Creating a very large executions store

In [7]:
import random
actorsList = []
for actors in range(2000):
    aid = 'DEV-DEVELOP_' + str(random.randint(1000000, 9999999))
    executionsList = []
    for execs in range(random.randint(100, 2000)):
        wid = random.randint(1000000, 9999999)
        eid = random.randint(1000000, 9999999)
        execDict = {'tenant': 'DEV-DEVELOP',
                    'api_server': 'https://dev.tenants.develop.tacc.cloud',
                    'actor_id': aid,
                    'executor': 'testuser',
                    'worker_id': wid,
                    'id': eid,
                    'message_received_time': '1585583279.07623',
                    'start_time': '1585583279.400957',
                    'runtime': 1,
                    'cpu': 239084680,
                    'io': 180,
                    'status': 'COMPLETE',
                    'exit_code': 0,
                    'final_state': {'Status': 'exited',
                                    'Running': False,
                                    'Paused': False,
                                    'Restarting': False,
                                    'OOMKilled': False,
                                    'Dead': False,
                                    'Pid': 0,
                                    'ExitCode': 0,
                                    'Error': '',
                                    'StartedAt': '2020-03-30T15:47:59.9001718Z',
                                    'FinishedAt': '2020-03-30T15:48:00.654149Z'}}
        executionsList.append(execDict)
    actorsDict = {'_id': aid,
                 'executions': executionsList}
    actorsList.append(actorsDict)
        
expandedList = second_stage_input
expandedList.extend(actorsList)
testDB['3'].drop()

In [8]:
testDB['3'].insert_many(expandedList)

## Archive

In [ ]:
# Example request
#'?search=car&exactsearch=DEV-DEVELOP&search=dog&&skip=0&limit=10'

# Mongo aggregation comparison expression operators to implement:
# $eq, $gt, %gte, $lt, $lte, $ne, $in, $nin

def get(search_type, args):
    """
    Placeholder
    """
    #args = dict(args.lists())    
    
    search_type = search_type.lower()
    search, query, paging = arg_parser(args)
    
    # Executions and Workers store
    if search_type == 'executions' or search_type =='workers':
        nested_search_pipeline(search, query, paging)
    elif search_type == 'actors' or search_type == 'logs':
        flat_search_pipeline(search, query, paging)
    else:
        raise KeyError(f'Inputted search_type is invalid, must be one of {list(store_dict.keys())}.')
    

def nested_search_pipeline(search, query, paging):
    print('-----PIPELINE-----\n')
    pprint.pprint(pipeline)
    print('\n\n\n-----RESULTANT-----')
    pprint.pprint(list(queried_store.aggregate(pipeline)))    

    
    

def get_derived_query_sections(search_type):
    queried_store, security, project = get_derived_pipeline_sections(search_type)
    """
    Placeholder
    """
    store_dict = {'executions': testDB['3'],
                  'workers': testDB['6'],
                  'actors': testDB['5'],
                  'logs': testDB['1']}
    try:
        queried_store = store_dict[search_type]
    except KeyError:
        raise KeyError(f'Inputted search_type is invalid, must be one of {list(store_dict.keys())}.')
    
    if search_type == 'executions':
        security = [{'$match': {'tenant': tenant}},
                    {'$lookup':
                        {'from' : '2',
                         'localField' : 'actor_id',
                         'foreignField' : '_id',
                         'as' : 'permissions'}},
                    {'$unwind': '$permissions'},
                    {'$match': {'permissions.' + curr_user: {'$exists': True}}}]
        project = [{'project': {'ijaiosjdioasjd'}}]
        
    elif search_type == 'workers':
        security = [{'$match': {'tenant': tenant}},
                    {'$lookup':
                        {'from' : '2',
                         'localField' : 'actor_id',
                         'foreignField' : '_id',
                         'as' : 'permissions'}},
                    {'$unwind': '$permissions'},
                    {'$match': {'permissions.' + curr_user: {'$exists': True}}}]
        project = [{'project': {'ijaiosjdioasjd'}}]
        
    elif search_type == 'actors':
        security = [{'$match': {'tenant': tenant}},
                    {'$lookup':
                        {'from' : '2',
                         'localField' : 'actor_id',
                         'foreignField' : '_id',
                         'as' : 'permissions'}},
                    {'$unwind': '$permissions'},
                    {'$match': {'permissions.' + curr_user: {'$exists': True}}}]
        project = [{'project': {'ijaiosjdioasjd'}}]
        
    elif search_type == 'logs':
        #Good
        security = [{'$match': {'tenant': tenant}},
                {'$lookup':
                    {'from' : '2',
                     'localField' : 'actor_id',
                     'foreignField' : '_id',
                     'as' : 'permissions'}},
                {'$unwind': '$permissions'},
                {'$match': {'permissions.' + curr_user: {'$exists': True}}}]
        project = [{'project': {'ijaiosjdioasjd'}}]
        
    return queried_store, security, project

def arg_parser(args):
    """
    Placeholder
    """
    query = []
    search = ""
    skip_amo = None
    limit_amo = None
    
    for key, val in args.items():
        if key == "search":
            if isinstance(val, list):
                joined_val = ' '.join(val)
                search = search + f' {joined_val}'
            else:
                search = search + f' {val}'
        elif key == "exactsearch":
            if isinstance(val, list):
                joined_val = '" "'.join(val)
                search = search + f' "{joined_val}"'
            else:
                search = search + f' "{val}"'
        elif key == "skip":
            if isinstance(val, list):
                raise KeyError('Inputted "skip" paramater is invalid, parameter can only be specified once')
            else:
                skip_amo = int(val)
        elif key == "limit":
            if isinstance(val, list):
                raise KeyError('Inputted "limit" paramater is invalid, parameter can only be specified once')
            else:
                limit_amo = int(val)
        else:
            used_oper = False
            for oper in ['.$eq', '.$gt', '.$gte', '.$lt', '.$lte', '.$ne']:
                if oper in key:
                    key = key.split(oper)[0]
                    oper = oper[1:]
                    query += [{'$match': {key: {oper: val}}}]
                    used_oper = True
                    break
            if not used_oper:
                query += [{'$match': {key: val}}]

    if search:
        search = search[1:] # getting rid of initial space
        query = [{'$match': {'$text': {'$search': search}}},
                 {'$sort': {'score': {'$meta': 'textScore'}}}] + query
    #pprint.pprint(query)
    
    paging = []
    if skip_amo:
        paging = paging + [{'$skip': skip_amo}]
    if limit_amo:
        paging = paging + [{'$limit': limit_amo}]
    #pprint.pprint(paging)

    return query, paging